**SA234 &#x25aa; Data Wrangling and Visualization &#x25aa; Spring 2026**

# Lesson 16. Split-Apply-Combine in Pandas

## In this lesson...

- One key task in data wrangling is to *split* our dataset into groups, *perform* a computation on each group, and *combine* the results

- This is often referred to as the **split-apply-combine** paradigm:
    * **Split** the data into groups based on some criteria
    * **Apply** a function to each group independently
    * **Combine** the results

- The apply step might involve one of the following:
    - aggregation
    - transformation
    - filtration

- Let's see how this works

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Setup

* Let's start by importing Pandas:

In [ ]:
import pandas as pd

* We'll use the nycflights13 dataset that we used in previous lessons, containing data on all flights outbound from NYC airports in 2013


* This data is located in `data/nycflights13_flights.csv.zip`, in the same folder as this notebook:

In [ ]:
df = pd.read_csv('data/nycflights13_flights.csv.zip')

* Just to remind ourselves what this dataset looks like:

In [ ]:
df.head()

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Aggregation

- **Aggregation** involves computing a summary statistic for each group of observations


- Visually:

<img src="img/aggregation.jpg" width="800"/>


- To make this concrete, let's work on an example with the nycflights13 dataset


- Suppose we want to explore the relationship between the distance traveled and arrival delay for each destination


- We can do the following:
    1. Group flights by destination
    2. Compute average distance, average arrival delay, and number of flights for each destination
    3. Filter to remove noise: only keep values for destinations with 20 or more flights, remove Honolulu

- *Quick aside.* Note that `.head(n)` returns the top `n` rows of a DataFrame
    - By default, `.head()` returns the top 5 rows

- Let's walk through this code, step-by-step


- First, `.groupby(['dest'])` *splits* the DataFrame `df` into groups of rows, according to the row's value of `dest`

- Next, `.agg(...)` *applies* various functions to each group independently

    - The keyword argument
    
        ```python
        avg_distance=('distance', 'mean')

        ```
        
        <br>outputs a variable called `avg_distance`, which is equal to the Pandas built-in function `mean` applied to the column `distance` *in each group*
        
    - The other keyword arguments above work in a similar fashion
    

    
- `.agg(...)` then takes the output from each group and *combines* them into a single output DataFrame, where the index is equal to the variables specified in `.groupby(...)`

- `.reset_index()` converts the existing index into ordinary columns, and resets the index of the DataFrame to the default one (consecutive integers)
    - This is often desired when performing additional wrangling or analysis steps

- `.query('(n_flights > 20) and (dest != "HNL")')` filters the rows of the output DataFrame

- **Basic template for aggregation:**

    ```python
    summarized_df = (
        df
        .groupby(list_of_variables)
        .agg(
            new_variable1=('variable1', 'aggregation function'),
            new_variable2=('variable2', 'aggregation function'),
            ...
        )
        .reset_index()
        ...
    )
    ```

- Here are some common aggregation functions
    - Note that many of them are *reduction methods* from Lesson 15
    
    
| Method | Description |
| :- | :- |
| `count` | Number of non-NA values |
| `first` | First value in group |
| `last` | Last value in group |
| `nunique` | Number of unique values |
| `min`, `max` | Minimum and maximum values |
| `sum` | Sum of values |
| `mean` | Mean of values |
| `median` | Median of values |
| `mad` | Mean absolute deviation from mean value |
| `prod` | Product of all values |
| `var` | Sample variance of values |
| `std` | Sample standard deivation of values |

- Instead of the `'aggregation function'` string, you can pass a function itself

    - For example, instead of `('variable1', 'sum')`, you could use<br><br>
    
        ```python
        ('variable1', lambda s: s.sum())
        ```
        
        <br>where `s` refers to the `variable1` column/Series of an *individual group* 

- *Note.* This method of using `.agg()` is called *named aggregation*
    - There are a few other ways of using `.agg()`
    - See the [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#aggregation) for details

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Grouping by multiple variables

- We can split a DataFrame into groups based on the values of *multiple* variables


- For example, we can compute the number of flights on each day like this:

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Transformation

- **Transformation** involves performing a computation on each group, and returning objects that are the same size as each group, with the same index


- Visually:

<img src="img/transformation.jpg" width="800"/>


- Transformation usually involves a "same size" method like we described in Lesson 15


- For example, recall that `.rank()` is a "same size" method that computes numerical ranks (for example, 1 = highest, 2 = second highest, etc.)


- For each day of the year, we can compute the rank of each flight according to its arrival delay, from highest (rank = 1) to lowest, like this:

<br>❓ **Exercise 1.** The first flight (with index 0) has the 296.5th highest arrival delay among all flights on the same day. What do you think a fractional rank means? 
*Hint.* [Here's the documentation for `.rank()`.](https://pandas.pydata.org/docs/reference/api/pandas.Series.rank.html)

*Write your notes here. Double-click to edit.*

- Let's walk through this code step-by-step


- `.groupby(['year', 'month', 'day'])` splits the DataFrame `df` into groups of rows, according to the combination of values of `year`, `month`, and `day`


- `['arr_delay']` then selects the `arr_delay` column/Series


- `.transform(...)` then takes the `arr_delay` Series *from each group* and applies the lambda function to it

- In `lambda s: s.rank(ascending=False)`:
    - `s` represents the `arr_delay` column/Series from each group
    - The lambda function outputs the numerical rank for each value in the Series, with the highest value having rank 1

- We can use the `.assign()` method to add these numerical ranks to our original DataFrame, like this:

❓ **Exercise 2.** Let's pick a random day of the year and check our work.
Filter the observations for all flights on June 12, 2013, and sort them in descending order of arrival delay. Display the top 10 flights. Do their values of `daily_arr_delay_rank` make sense?

* **Basic template for transformation:**

    ```python
    df_with_transformed_column = df.assign(
        new_variable=lambda x: 
            x.groupby(list_of_variables)
            ['variable']
            .transform(lambda s: s.same_size_method(...))
    )
    ```
    
    <br>where `x` refers to `df`, and `s` refers to `x['variable']`

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Filtration

- **Filtration** allows us to keep or discard some groups based on a group-wise computation that evaluates to `True` or `False`


- Visually:

<img src="img/filtration.jpg" width="800"/>


- For example, we can keep all flights to destination airports with 10,000 or more total flights, like this:

- How does this code work?


- First, `.groupby(['dest'])` splits the DataFrame `df` into groups of rows, according to the row's value of `dest`


- Then, `.filter(...)` takes a function that, when applied to the group as a whole, returns True or False

- In `lambda x: x['flight'].count() > 10000`:
    - `x` represents a single group
    - `x['flight'].count()` counts the non-NA values in the `flight` column of each group `x`

❓ **Exercise 3.** Let's check our work. Using the techniques from the *Aggregation* section above, count the number of flights to each destination in `busy_dest_df`. Does what you see make sense?

* **Basic template for filtration:**

    ```python
    filtered_df = (
        df
        .groupby(list_of_variables)
        .filter(lambda x: expression that evaluates to True or False)
    )
    ```
    
    <br>where `x` refers to `df`

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Related tips and tricks

### Aggregation on the entire DataFrame 

- As we've seen above, the split-apply-combine paradigm gives us a nice, standard template for computing summary statistics for groups of observations in a DataFrame (i.e., aggregation)


- We can also use the same template for computing summary statistics for *all* the observations in a DataFrame 


- Suppose we want the average, median, minimum, and maximum arrival delay across all flights in 2013


- We can do so like this:

- A few variations from the standard aggregation template...


- We give `.groupby()` the argument `lambda r: True` to put all the rows into one group 

- Why does this work? 
    - `.groupby()` can take a function `lambda r: ...` as an argument, where `r` represents a row index
    - When given such an argument, `.groupby()` forms groups according to the output values of this lambda function
    - In the code above, *every* row index returns the value `True`, so every row gets put into the same group 

- We also use `drop=True` in `.reset_index()`, since we don't need the name of the group in this case

### Counting unique values and `.value_counts()`

- We can use the above techniques to count the unique values in a particular column


- For example, we can count the number of flights by each airline like this:

- You can also get a similar table of counts using the `.value_counts()` method, like this:

- Using `.value_counts()` can be quick and convenient when all you need is to count the number of unique values in a column/Series


- If you need additional summary statistics, the standard aggregation template using `.groupby(...).agg(...)` may be easier to use

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Problems

For the problems below, use the nycflights13 dataset that we used in this lesson.

### Problem 1

What time of day should you fly if you want to avoid delays as much as possible? Compute the average departure and arrival delays for each hour of the day.

### Problem 2

Assume that flights with missing arrival delays correspond to cancelled flights. Compute the number of cancelled flights for each month.

*Hint.* Use `.query()` to narrow down the dataset to rows corresponding to cancelled flights.

### Problem 3

Which major carrier has the worst delays? Compute the average arrival delay for each carrier that has more than 10,000 flights in 2013. Sort them from highest to lowest average arrival delay.

### Problem 4

Perhaps it's a better idea to compare the average arrival delay between carriers on the same route.

Focus on routes from a NYC airport (EWR, JFK, LGA) to either Atlanta (ATL) or Chicago O'Hare (ORD). For each route, find the carrier with the highest average arrival delay.

*Hint.* Use `.agg()`, sort the resulting values, and then use `.agg()` again.

### Problem 5

How often does a plane (tail number) leave on time? For each plane, compute the fraction of flights it leaves on time.

_Hint._ Define a function `on_time(delay)` that takes the delay as input, and outputs 1 if the delay is nonpositive (the flight is on time), and 0 otherwise. Use this function with `.map()` &ndash; see Lesson 15 for details. 

### Problem 6

For each plane (tail number), compute its cumulative departure delay over each day in 2013 that it departed from a NYC airport.

*Hint.* First, remove all flights with missing departure delays.

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Notes and sources

- From the [Pandas User Guide](https://pandas.pydata.org/docs/user_guide/index.html):
    - [Group by: split-apply-combine](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html)

- Lesson and problems inspired by Chapter 5 of [R for Data Science](https://r4ds.had.co.nz/)